# 1.Extracting data from the website

In [1]:
# Import packages
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
# Using https://avherald.com/

# Retrieve links of pages excluding website announcements

base_url = "https://avherald.com"
wo_news = "/h?list=&opt=2048"


headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 11.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36"
}

base_response = requests.get(base_url+wo_news,headers = headers )

if base_response.status_code == 200:
    soup = BeautifulSoup(base_response.text,'html.parser')
else:
    print("Fetch page failed")

In [3]:
# Extract links to wikibase
import re
ids = []
for s in soup.find_all('a'):
    link = s.get('href')
    pattern = r'(\/h\?article=\w+)'
    match = re.findall(pattern, link)
    if match:
        ids.append(match[0])
ids
print(f"Page X imported")

Page X imported


In [4]:
ids

['/h?article=52439b47',
 '/h?article=523ff442',
 '/h?article=52374362',
 '/h?article=52418266',
 '/h?article=52417dcb',
 '/h?article=52417ba6',
 '/h?article=52416bbe',
 '/h?article=523b782d',
 '/h?article=524168c3',
 '/h?article=52416727',
 '/h?article=523ec753',
 '/h?article=51b7473c',
 '/h?article=5240d916',
 '/h?article=5018aa32',
 '/h?article=5240ba0d',
 '/h?article=5240abca',
 '/h?article=5240aec2',
 '/h?article=524027d7',
 '/h?article=52401f3c',
 '/h?article=52401694',
 '/h?article=524001d2',
 '/h?article=523f91e8',
 '/h?article=523f882f',
 '/h?article=523f827b',
 '/h?article=523ece13',
 '/h?article=523ecbf0',
 '/h?article=523ec3d3',
 '/h?article=514b23d5',
 '/h?article=523eb568',
 '/h?article=523e19b9',
 '/h?article=523ceeec',
 '/h?article=523cedc0',
 '/h?article=523cd65e',
 '/h?article=523cd3b8',
 '/h?article=523c3d01',
 '/h?article=523c1d8d',
 '/h?article=521fd4fb',
 '/h?article=523ba0ee',
 '/h?article=523adfe4',
 '/h?article=523add12',
 '/h?article=523ad9f3',
 '/h?article=523

In [56]:
# Accessing individual page

page_response = requests.get(base_url+ids[0], headers = headers)

soup = BeautifulSoup(page_response.text, 'html.parser')

In [57]:
base_url+ids[0]

'https://avherald.com/h?article=52439b47'

In [58]:
headline = soup.find_all('span', class_ = 'headline_article')[0].text
headline

'Accident: Endeavor CRJ9 at Toronto on Feb 17th 2025, flipped over on landing'

In [92]:
accident_type = re.findall(r'(\w+):', headline)[0]

In [120]:
# Extract text in question, with tags
body = soup.find_all('span', class_ = 'sitetext')[3]
body_text = "".join(str(b) for b in body.contents)

# Split the text into chunks
body_text = body_text.split('<br/><br/>')

# Remove text chunks with external links as they are not useful for the analysis
body_text = [chunk for chunk in body_text if 'http' not in chunk]
body_text = "\n\n".join(body_text)

# Clean br tags
body_text = re.sub(r'<br\s*/?>','\n',body_text)

body_text

'An Endeavor Airlines Canadair CRJ-900 on behalf of Delta Airlines, registration N932XJ performing flight DL-4819 from Minneapolis,MN (USA) to Toronto,ON (Canada) with 76 passengers and 4 crew, landed on Toronto\'s runway 23 at 14:12L (19:12Z) but came to a stop on the runway inverted, both wings, both main gear and the tailplane separated from the airframe. There were no fatalities, however, one person was critically injured, 7 others received minor injuries.\n\nToronto Airport reported all people on board have been accounted for. \n\nEmergency services reported one person is in critical condition, 7 others have received injuries.\n\nThe Canadian TSB reported: "TSBAir is deploying a team to investigate an aircraft accident at Toronto / Lester B. Pearson International Airport, ON. The TSB will gather information and assess the occurrence."\n\nThe NTSB reported: "The NTSB is leading a team of U.S. investigators to assist the Transportation Safety Board of Canada with their investigation

In [121]:
date_pattern = r'\b(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s\d{1,2}(?:st|nd|rd|th)?,?\s\d{4}\b'
ext_date = re.findall(date_pattern, headline)[0]

In [122]:
clean_date = re.sub(r'(st|nd|rd|th)', '', ext_date)

In [123]:
from datetime import datetime

formatted_date = datetime.strptime(clean_date, "%b %d %Y").strftime('%Y-%m-%d')
formatted_date

'2025-02-17'

In [124]:
filter = r'registration (\w+)'
registration_code = re.findall(filter, body_text)[0]

In [125]:
formatted_date+'_'+registration_code

'2025-02-17_N932XJ'

In [126]:
import os

file_path = os.path.join(os.getcwd(),'files')
file_path

'/home/kwonowk/code/kwonowk/Projects/aviation-safety/files'

In [127]:
file_name = formatted_date+'_'+accident_type+'_'+registration_code+'.txt'
with open('files/'+file_name, 'w') as f:
    f.write(headline)
    f.write('\n')
    f.write(body_text)

# Extract detailed information from the text
Using Chat GPT

- Type of event : crash / accident / incident 
- operator : 
- marketing carrier : 
- plane Registration code
- Datetime
- Flight code 
- Plane model
- from_airport : 
- from_city :
- from_country :
- to_city : 
- to_country : 
- to_airport : 
- onboard_passengers :
- onboard_crews
- fatalities
- injuries_critical :
- injuries_minor :
- injuries_total : 
- cause : plane / external 
- investigation_status 


In [ ]:
from openai import OpenAI
client = OpenAI()
completion = client.chat.completions.create(
    model="gpt-4o-mini-2024-07-18",
    store=True,
    messages=[
        {"role": "user", "content": "write a haiku about ai"},

    ]
)

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable